In [23]:
def clean_and_tokenize_text(text):
    """
    Cleans `text` from punctuation and special symbols
    """    
    # need to adjust this for better cleaning
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords

    lower_alpha_tokens = [w for w in word_tokenize(text.lower()) if w.isalpha()]
    no_stop = [t for t in lower_alpha_tokens if t not in set(stopwords.words('english'))]
    
    wnl = WordNetLemmatizer()
    lemmatized = [wnl.lemmatize(t) for t in no_stop]
    return lemmatized

def quality_check(article):
    """
    Checks for quality of the `article` and returns False if it doesnt' pass the 
    QA check
    """
    
    # need to find ways for better QA
    if article and len(article) > 150: return True
    return False

def concat_news(ticker, period=1):
    """
    Reads all news from the `DB` for the `ticker`, 
    concats by the `period`.
    
    Input
    -----
    period: int, default 1 (day)
    
    Output
    ------
    resulting_dictionary: dict, key–day, value–all news for that day
    Counter(all_tokens)
    """
    from collections import Counter
    
    news = collection.find_one({
        'ticker' : ticker
    })['news']
    
    resulting_dictionary = {}
    
    all_tokens = []
    
    for n in news:
        try:
            date = str(pd.to_datetime(n['datetime']).date())

            tokens = clean_and_tokenize_text(n['text'])

            text = ' '.join(tokens)

            all_tokens.extend(tokens)

            if quality_check(text):
                if date in resulting_dictionary.keys():
                    resulting_dictionary[date] = resulting_dictionary[date] + text
                else:
                    resulting_dictionary[date] = text
        except Exception as e:
            print(f"Cannot process url: {n['url']}\nError: {e}")
    
    return resulting_dictionary, Counter(all_tokens)

In [ ]:
from yahoo_fin import stock_info as si 
import pymongo as pm
import pandas as pd
from datetime import date

# create dataset for predicting pricing for a company

# set the company ticker
ticker = 'AAPL'

# set interval
start_date = '01/01/2019'
end_date = date.today()

# obtain historical stock pricing data
t_data = si.get_data(ticker, start_date=start_date, end_date=end_date)

# Start MongoDB
# !brew services start mongodb-community@4.2

# Stop MongoDB
# !brew services stop mongodb-community@4.2

# connect to DB
client = pm.MongoClient('mongodb://localhost:27017')
collection = client['news']['recommendations']

# get the dictionary with all news per period (1 day) and BOW (bag of words)
t_news, t_bow = concat_news(ticker, period=1)

# convert to pandas df
t_df = pd.DataFrame.from_dict(d, orient='index', columns=['text'])
t_df.index = pd.to_datetime(t_df.index)

In [67]:
# combine news with stock price 'adjclose'
t_combined = pd.concat([t_df, t_data.adjclose], axis=1)

# fill adjclose for over the weekends and holiday
# logic is the price on monday close is the result of 
# the news over the weekend hence backfill
t_combined.adjclose = t_combined.adjclose.fillna(method='backfill')

# drop rows without the stock price
t_combined = t_combined[t_combined.adjclose.notna()]
t_combined.tail(20)

,text,adjclose
2020-06-10,enthusiasm seems waned following powerful rall...,352.839996
2020-06-11,bloomberg apple analyst grown positive company...,335.899994
2020-06-12,technology best performing sector sector emerg...,338.799988
2020-06-13,dow jones industrial average continued rally s...,342.989990
2020-06-14,warren buffett trade portfolio put protect dow...,342.989990
2020-06-15,secret retail investor stepping participation ...,342.989990
2020-06-16,douglas busvine berlin reuters germany sought ...,352.079987
2020-06-17,earlier week apple nasdaq aapl released result...,351.589996
2020-06-18,printed google logo placed apple macbook illus...,351.730011
2020-06-19,two main headline berkshire hathaway nyse nyse...,349.720001


In [69]:
t_combined.to_csv(f'to_predict/{ticker}.csv')

In [65]:
t_combined.adjclose.dropna().tail(20)

2020-06-10    352.839996
2020-06-11    335.899994
2020-06-12    338.799988
2020-06-13    342.989990
2020-06-14    342.989990
2020-06-15    342.989990
2020-06-16    352.079987
2020-06-17    351.589996
2020-06-18    351.730011
2020-06-19    349.720001
2020-06-20    358.869995
2020-06-21    358.869995
2020-06-22    358.869995
2020-06-23    366.529999
2020-06-24    360.059998
2020-06-25    364.839996
2020-06-26    353.630005
2020-06-27    361.779999
2020-06-28    361.779999
2020-06-29    361.779999
Name: adjclose, dtype: float64